<center>
    
    Toy dataset with CNN
    
    Author: Daniel Coble
    
    Status: Work in progress
</center>

In "Toy datasets and training an MLP", I created a simple toy dataset with entries of 32x32 black and white pixel grids. Each grid had two lines, and it was the goal of the neural net to predict where the lines would intersect. One can easily see why an MLP might not be the best choice for this type of data. For one thing, the MLP has no sense of left, right, up, and down. It recieves only a flattened vector of the pixel grid, and must intuit our idea of space from the training data. (Imagine all the pixels in the dataset were randomized before being handed to you, and you didn't know how they were randomized. You would find thed task much harder.)

Convolutional neural networks do have a sense of direction. The 2D convolutional network trained here applies the same convolution (technically a ) to every element of the 2D grid input to produce another 2D grid. With progressive layers CNNs, the grid will get smaller by the use of __.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import math
import random
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import losses
from math import floor
from sklearn import model_selection


def createplot():
    length = .3
    length_sq = length**2
    point = [random.random(), random.random()]
    x1 = random.random()
    y1 = random.random()
    no_space_line_1 = True
    while(no_space_line_1):
        no_space_line_1 = False
        dist1 = (x1-point[0])**2 + (y1-point[1])**2
        if(dist1 < length_sq):
            no_space_line_1 = True
            x1 = random.random()
            y1 = random.random()
    x2 = random.random()
    y2 = random.random()
    no_space_line_2 = True
    while(no_space_line_2):
        no_space_line_2 = False
        dist2 = (x2-point[0])**2 + (y2-point[1])**2
        if(dist2 < length**2):
            no_space_line_2 = True
            x2 = random.random()
            y2 = random.random()
    
    dist1 = math.sqrt(dist1)
    dist2 = math.sqrt(dist2)
    lineseg_1 = np.array([[x1,y1],[x1 + length*(point[0] - x1)/dist1,y1 + length*(point[1] - y1)/dist1]])
    lineseg_2 = np.array([[x2,y2],[x2 + length*(point[0] - x2)/dist2,y2 + length*(point[1] - y2)/dist2]])
    lineseg_1 = 32*lineseg_1
    lineseg_2 = 32*lineseg_2
    delta_x = 0.01 * (lineseg_1[1,0] - lineseg_1[0,0])
    delta_y = 0.01 * (lineseg_1[1,1] - lineseg_1[0,1])
    x1 *= 32
    y1 *= 32
    
    plot = np.zeros(shape=(32,32))
    for i in range(100):
        x_index = floor(x1 + i*delta_x)
        y_index = floor(y1 + i*delta_y)
        plot[x_index, y_index] += .0678822
    
    x2 *= 32
    y2 *= 32
    delta_x = 0.01 * (lineseg_2[1,0] - lineseg_2[0,0])
    delta_y = 0.01 * (lineseg_2[1,1] - lineseg_2[0,1])
    
    for i in range(100):
        x_index = floor(x2 + i*delta_x)
        y_index = floor(y2 + i*delta_y)
        plot[x_index, y_index] += .0678822
    
    return (plot, point)
    
        

amount = 100000
data = []
for i in range(amount):
    if((i+1) % (amount//10) == 0):
        print(str(round((i+1) / amount * 100)) + "% finished.")
    data.append(createplot())
plots = np.array([data[i][0] for i in range(amount)])
points = np.array([data[i][1] for i in range(amount)])

train_img, test_img, train_label, test_label = model_selection.train_test_split(plots, points, test_size=.2)

for i in range(4):
    randint = random.randint(0,2000)
    true = test_label[randint] 
    plt.figure()
    plt.imshow(test_img[randint], cmap=mpl.cm.binary, interpolation = 'nearest', origin='lower', extent=[0,1,0,1])
    plt.grid(False)
    plt.plot(true[1], true[0], marker='x', markersize=12, color="red")
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import losses


#train_img = np.load("./dataset/train_img.npy")
#train_label = np.load("./dataset/train_label.npy")
#test_img = np.load("./dataset/test_img.npy")
#test_label = np.load("./dataset/test_label.npy")

model = keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

model.fit(train_img, train_label, epochs=20)

I said that the advantage of CNNs is that they have very few weights. Compared to a dense layer, a CNN is sparsely connected and uses weight sharing to further reduce the amount of trainable weights. The MLP we trained had 34,978 weights, while this CNN has 

In [ ]:
train_loss, train_acc = model.evaluate(train_img,  train_label, verbose=0)
test_loss, test_acc = model.evaluate(test_img, test_label, verbose=0)
print('Training accuracy:', train_acc)
print('Test accuracy: ', test_acc)

weight_sum = 0
for weights in model.trainable_weights:
    weight_sum += np.size(weights)
print("Amount of weights: ", weight_sum)
print("Training set size: ", train_img.shape[0])